In [1]:
import osmnx as ox
import pandas as pd
import folium
import matplotlib
import numpy as np
import geopandas as gpd 
from ipyleaflet import Map, GeoData

## Fitness Centres - Centroids and Building Footprints
"Fitness centre, health club or gym with exercise machines, fitness classes or both, for exercise."

In [2]:
place = "Spokane County, Washington"

gym_tags = {'leisure':'fitness_centre'}
bldg_tags = {"building":True}

gdf_gym = ox.features_from_place(place, gym_tags)
gdf_bldgs = ox.features_from_place(place, bldg_tags)

In [3]:
gdf_gym.explore()

the result is a GeoDataFrame

In [4]:
print(gdf_bldgs.shape)
print(type(gdf_bldgs))
print(gdf_bldgs.crs)

(148999, 317)
<class 'geopandas.geodataframe.GeoDataFrame'>
epsg:4326


In [5]:
# keep only footprints (polygons)
gdf_bldgs = gdf_bldgs[gdf_bldgs.geometry.geom_type.isin(['Polygon'])]
gdf_bldgs.shape

(148950, 317)

In [6]:
print(gdf_gym.shape)
print(gdf_gym.crs)

(61, 26)
epsg:4326


## Are there buildings tagged with 'fitness'?

In [7]:
sorted(gdf_bldgs['amenity'].unique()[1:])

['animal_boarding',
 'animal_breeding',
 'animal_shelter',
 'arts_centre',
 'bank',
 'bar',
 'beauty_school',
 'cafe',
 'car_rental',
 'car_wash',
 'casino',
 'childcare',
 'cinema',
 'clinic',
 'community_centre',
 'courthouse',
 'dancing_school',
 'dentist',
 'doctors',
 'dojo',
 'dressing_room',
 'driving_school',
 'events_venue',
 'fast_food',
 'fire_station',
 'fraternity',
 'fuel',
 'ice_cream',
 'kindergarten',
 'library',
 'marketplace',
 'motel',
 'music_school',
 'parking',
 'pharmacy',
 'place_of_mourning',
 'place_of_worship',
 'police',
 'post_depot',
 'post_office',
 'pub',
 'public_building',
 'ranger_station',
 'recycling',
 'restaurant',
 'retirement_home',
 'school',
 'shelter',
 'shower',
 'social_centre',
 'social_club',
 'social_facility',
 'studio',
 'theatre',
 'toilets',
 'townhall',
 'veterinary']

### ? include additional POIs ('dojo', 'dancing_school', etc.)

In [8]:
# there are only four 
gdf_bldgs['amenity'].isin(['dojo', 'dancing_school']).sum()

4

### actually, some of the points are shapes!

In [9]:
gdf_gym['geometry'].geom_type.value_counts()

Point      37
Polygon    24
Name: count, dtype: int64

## Goal: separate gym points from gym polygons 

In [10]:
gdf_gym.geometry.geom_type.value_counts()

Point      37
Polygon    24
Name: count, dtype: int64

separate points from polygons

In [11]:
gym_points = gdf_gym[gdf_gym.geometry.geom_type.isin(['Point'])]
gym_polygons = gdf_gym[gdf_gym.geometry.geom_type.isin(['Polygon'])]    

get buildings that intersect with the points

In [22]:
print(gym_polygons.columns.tolist())
print(gym_polygons['building'].value_counts(dropna = False))

['element_left', 'id_left', 'geometry', 'addr:city_left', 'addr:housenumber_left', 'addr:state_left', 'addr:street_left', 'building_left', 'name_left', 'addr:postcode_left', 'building:levels_left', 'shop', 'source', 'roof:levels', 'roof:shape', 'element_right', 'id_right', 'leisure_right', 'name_right', 'sport_right', 'brand_right', 'brand:wikidata_right', 'opening_hours_right', 'wheelchair_right', 'addr:city_right', 'addr:housenumber_right', 'addr:postcode_right', 'addr:state_right', 'addr:street_right', 'addr:unit_right', 'branch_right', 'check_date_right', 'phone_right', 'website_right', 'ref_right', 'leisure', 'name', 'sport', 'brand', 'brand:wikidata', 'opening_hours', 'addr:city', 'addr:housenumber', 'addr:postcode', 'addr:state', 'addr:street', 'check_date', 'phone', 'website', 'ref', 'building', 'building:levels', 'construction', 'access', 'fixme:atp', 'email', 'operator']
building
NaN             37
yes             19
retail           2
school           1
construction     1
un

In [30]:
gym_polygons = gym_polygons[gym_polygons['building']=='construction']

In [13]:
# keep all points (centroids)
bldgs_contain_gym_points = gdf_bldgs.sjoin(gym_points, how="inner", predicate="contains")

# confirm 1:1 match
print(bldgs_contain_gym_points.shape)
print(gym_points.shape)

(37, 344)
(37, 26)


### Combine the two types of buildings data

In [14]:
# fix multi-index
bldgs_contain_gym_points.reset_index(inplace=True)

In [15]:
# confirm same crs
bldgs_contain_gym_points.crs == gym_polygons.crs

True

In [16]:
gdf_bldgs["amenity"].isna().sum()

147088

In [17]:
gym_polygons = gpd.GeoDataFrame(pd.concat([bldgs_contain_gym_points, gym_polygons], ignore_index=True), crs=bldgs_contain_gym_points.crs)

### most columns have all NaN values

In [18]:
gym_polygons = gym_polygons.dropna(axis=1, how='all')

## Explore results

- We have 61 gym building footprints, and 37 gym points 
- All 37 gym points have an associated building footprint

In [19]:
print(gym_polygons.shape)
print(gym_points.shape)

(61, 57)
(37, 26)


## Plot interactively

In [29]:
from branca.element import Template, MacroElement

m = folium.Map(location=[gym_points.geometry.centroid.y.mean(), gym_points.geometry.centroid.x.mean()], zoom_start=15, tiles=None)

# Add two (2) basemap layers
folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Esri',
    name='ESRI Aerial Imagery',
    overlay=False,
    control=True
).add_to(m)

folium.TileLayer('CartoDB Positron', name = "Positron", control=True).add_to(m)

# assign colors dynamically
# types = [e for e in sports_gdf['leisure'].unique() if pd.notna(e)] + ["school", "parking"]  # skip nan\
# colors = matplotlib.cm.get_cmap('viridis', len(types))
# color_dict = {t:matplotlib.colors.rgb2hex(colors(i)) for i, t in enumerate(types)}
color_dict = {'polgon': 'red', 'point':'blue'}

# Create feature groups for each type and add to map
feature_groups = {}
for t in ['polygon', 'point']:
    feature_groups[t] = folium.FeatureGroup(name=t).add_to(m)



# Function to add polygons to the map
def add_polygons(gdf, map_object):
    for _, row in gdf.iterrows():

        ''' 
        # assign type and color
        if 'leisure' in row.keys() and pd.notna(row['leisure']):
            item_type = row['leisure']
            curr_color = color_dict[row['leisure']]
        elif pd.notna(row['amenity']) and row['amenity'] == 'parking':
            item_type='parking'
            curr_color = color_dict['parking']
        else:
            item_type = 'school'
            curr_color = color_dict['school']
        '''

        item_type = 'polygon'
        curr_color = 'red'

        if pd.notna(row['name']):
            popup_name = row['name']
        else:
            popup_name = "unnamed"
        
        # python's late binding behavior with lambdas!
        sim_geo = folium.GeoJson(row.geometry, 
                                style_function=lambda x, color=curr_color: {'fillColor': color, 'color': color}, control=False)
        
        # Check for non-null and convert name to string
        #if pd.notna(row['name']):
        #    folium.Popup(str(row['name'])).add_to(sim_geo)

        folium.Popup(f"<strong>{popup_name.capitalize()}</strong> \n\n ").add_to(sim_geo)

        sim_geo.add_to(feature_groups[item_type])

# Function to add points to the map
# these are all schools
def add_points(gdf, map_object):

    for _, row in gdf.iterrows():

        # assign type and color
        '''
        if 'leisure' in row.keys() and pd.notna(row['leisure']):
            item_type = row['leisure']
            curr_color = color_dict[row['leisure']]
        else:
            item_type = 'school'
            curr_color = color_dict['school']
        '''

        item_type = 'point'
        curr_color = 'blue'

        if pd.notna(row['name']):
            popup_name = row['name']
        else:
            popup_name = "unnamed"

        folium.CircleMarker(
            [row.geometry.y, row.geometry.x],
            popup=f"<strong>{popup_name.capitalize()}</strong> \n\n ",  # Assuming you have a 'name' column for the popup
            color=curr_color, 
            fill=True, 
            fill_color=curr_color,
            fill_opacity=0.6,
            radius=2,
            control=False
        ).add_to(feature_groups[item_type])

# add the data to the map
add_polygons(gym_polygons, m)
add_points(gym_points, m)


# add a legend
template = """
{% macro html(this, kwargs) %}
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 150px; height: auto; 
            border:2px solid grey; z-index:9999; font-size:14px;
            background-color:white; padding: 5px;">
    <p><strong>Legend</strong></p>
    {% for key, color in this.color_dict.items() %}
    <p><span style="color: {{color}}">&#11044;</span> {{key}}</p>
    {% endfor %}
</div>
{% endmacro %}
"""

macro = MacroElement()
macro._template = Template(template)
macro.color_dict = color_dict
m.get_root().add_child(macro)

# add a title
map_title = "OpenStreetMap Fitness Center Features (Spokane County, WA | 10-05-2025)"
title_html = f'<h1 style="position:absolute;z-index:100000;left:40vw" ><strong>{map_title}</strong></h1>'
m.get_root().html.add_child(folium.Element(title_html))

folium.LayerControl(collapsed=False, position="bottomright").add_to(m)

# Display the map
display(m)
m.save("Fitness_Centers_Spokane.html")

/var/folders/qx/lhjn1vkn095_xhkfhcmh5n140000gn/T/ipykernel_62757/3300670807.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  m = folium.Map(location=[gym_points.geometry.centroid.y.mean(), gym_points.geometry.centroid.x.mean()], zoom_start=15, tiles=None)


### Save to disk 

for gdf_obj, layer_name in [(mso_parks, "parks"), (mso_pitches, "pitches"), 
                            (mso_sports_centres, "sports_centres"), (mso_schools, "schools")]:

    # convert to a feet-based crs 
    gdf_obj.to_crs(epsg=2256, inplace=True)

    # save as layer in a GeoPackage
    gdf_obj.to_file("mso_features.gpk", layer=layer_name, driver="GPKG")

In [21]:
gym_points.to_file("Fitness_Centers_Spokane.gpk", layer="points", driver="GPKG")    
gym_polygons.to_file("Fitness_Centers_Spokane.gpk", layer="polygons", driver="GPKG")

/opt/anaconda3/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: The filename extension should be 'gpkg' instead of 'gpk' to conform to the GPKG specification.
  ogr_write(
/opt/anaconda3/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: File Fitness_Centers_Spokane.gpk has GPKG application_id, but non conformant file extension
  ogr_write(
